In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from math import sin, cos, sqrt, atan2, radians
from geopy.distance import vincenty
from sklearn import preprocessing
from scipy import interpolate
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
import networkx as nx
# First, import the data:
pd.options.display.max_columns = 100
# pd.options.display.max_rows = None


## Adding a month of data - Jan 2016 - Feb 2016


list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116',
                    'fahrzeiten_soll_ist_20160117_20160123', 'fahrzeiten_soll_ist_20160124_20160130',
                    'fahrzeiten_soll_ist_20160131_20160206', 'fahrzeiten_soll_ist_20160207_20160213',
                    'fahrzeiten_soll_ist_20160214_20160220', 'fahrzeiten_soll_ist_20160221_20160227',
                    'fahrzeiten_soll_ist_20160228_20160305', 'fahrzeiten_soll_ist_20160306_20160312',
                    'fahrzeiten_soll_ist_20160313_20160319', 'fahrzeiten_soll_ist_20160320_20160326']

# Just the first two weeks

# list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116']

list_of_df = []
for i in list_of_df_names:
    list_of_df.append(pd.read_csv(i+'.csv'))
df_1 = pd.concat(list_of_df, ignore_index=True)
list_of_df.clear()


####################
df_haltepunkt = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltepunkt.csv")
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
# df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
# df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


# Bus Station
df_haltestelle = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltestelle.csv")
####################

def find_dist(lat1, lon1, lat_2, lon2):
    c1 = (lat1, lon1)
    c2 = (lat_2, lon2)
    return geopy.distance.vincenty(c1, c2).km
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################


In [2]:
# Drop the punkts with undefined gps data
df_haltepunkt.dropna(axis=0, how='any', inplace=True, subset=['GPS_Longitude'])


In [3]:
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


In [4]:
# Number of records
len(df_1)

16825506

In [5]:
# Merge the halt punkt from


df_1['halt_punkt_id'] = df_1['halt_punkt_id_von']
df_1 = df_1.merge(df_haltepunkt, on=['halt_punkt_id'])


In [6]:
# df_1['GPS_Latitude_from'] = df_1['GPS_Latitude']
# df_1['GPS_Longitude_to'] = df_1['GPS_Longitude']

# Merge punkt to

df_1['halt_punkt_id'] = df_1['halt_punkt_id_nach']
df_1 = df_1.merge(df_haltepunkt, on='halt_punkt_id')


In [7]:

df_1['halt_id'] = df_1['halt_id_x']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [8]:
df_1['halt_id'] = df_1['halt_id_y']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16825506 entries, 0 to 16825505
Data columns (total 56 columns):
linie                     int64
richtung                  int64
betriebsdatum             object
fahrzeug                  int64
kurs                      int64
seq_von                   int64
halt_diva_von             int64
halt_punkt_diva_von       int64
halt_kurz_von1            object
datum_von                 object
soll_an_von               int64
ist_an_von                int64
soll_ab_von               int64
ist_ab_von                int64
seq_nach                  int64
halt_diva_nach            int64
halt_punkt_diva_nach      int64
halt_kurz_nach1           object
datum_nach                object
soll_an_nach              int64
ist_an_nach1              int64
soll_ab_nach              int64
ist_ab_nach               int64
fahrt_id                  int64
fahrweg_id                int64
fw_no                     int64
fw_typ                    int64
fw_kurz         

In [10]:
# delete the entries where the days are not matching
df_1 = df_1[df_1['betriebsdatum'] == df_1['datum_nach']]

In [11]:
# Some Signals
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'
delay = 'delay'

actual_stationary = 'actual_stationary'
actual_travel_time = 'actual_travel_time'

df_1[target_stationary] = df_1[target_departure_from] - df_1[target_arrival_from]
df_1[target_travel_time] = df_1[target_arrival_to] - df_1[target_departure_from]
df_1[delay] = df_1[actual_arrival_from] - df_1[target_arrival_from]


# Signal - 1 - actual arrival - actual departure
df_1[actual_stationary] = df_1[actual_departure_from] - df_1[actual_arrival_from]
df_1[actual_travel_time] = df_1[actual_arrival_to] - df_1[actual_departure_from]

# b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
# b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

# Merge halts - Define some attr.

In [12]:
# Creating a pair of gps locations and calculating the signal at that time instance:
# temp['Road_Segment_from'] = list(zip(temp.GPS_Latitude_x, temp.GPS_Longitude_x))
# temp['Road_Segment_to'] = list(zip(temp.GPS_Latitude_y, temp.GPS_Longitude_y))
# temp

# aggregating the gps locations of the same punkts
df_new_x = df_1.groupby(['halt_id_x'], as_index=False)['GPS_Latitude_x', 'GPS_Longitude_x'].mean()
df_new_y = df_1.groupby(['halt_id_y'], as_index=False)['GPS_Latitude_y', 'GPS_Longitude_y'].mean()
# Merging the two frames signifying a road segment
final = df_1.merge(df_new_x, on=['halt_id_x'])
final = final.merge(df_new_y, on=['halt_id_y'])


In [13]:
final.rename(columns={'GPS_Latitude_x_y':'GPS_LAN_STELLE_FROM', 'GPS_Longitude_x_y':'GPS_LON_STELLE_FROM',
                      'GPS_Latitude_y_y':'GPS_LAN_STELLE_TO', 'GPS_Longitude_y_y':'GPS_LON_STELLE_TO'}, inplace=True)
# final

In [14]:
# To release memeory
df_1 = 1

In [15]:
# If This treats the roads as punkt to punkt
# final['Road_Segment_From'] = list(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x))
# final['Road_Segment_To'] = list(zip(final.GPS_Latitude_y_x, final.GPS_Longitude_y_x))
# final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
# final['Route_Node_id_tuple'] = list(zip(final.halt_punkt_id_von, final.halt_punkt_id_nach))


## The following treats the road segments as haltestelle to haltestelle
final['Road_Segment_From'] = list(zip(final.GPS_LAN_STELLE_FROM, final.GPS_LON_STELLE_FROM))
final['Road_Segment_To'] = list(zip(final.GPS_LAN_STELLE_TO, final.GPS_LON_STELLE_TO))
final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
final['Route_Node_id_tuple'] = list(zip(final.halt_id_x, final.halt_id_y))
final.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple
0,918,1,06.01.16,11652,1,2,3744,50,ZENK,06.01.16,65880,65906,65898,65906,3,2043,50,RKIK,06.01.16,65988,65997,66000,65997,75165,5091,1,1,1,BKUE - GART,112395,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,26,0,91,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)"
1,918,1,06.01.16,11664,2,2,3744,50,ZENK,06.01.16,24480,24561,24492,24572,3,2043,50,RKIK,06.01.16,24576,24657,24588,24657,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,81,11,85,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)"
2,918,1,09.01.16,11655,2,2,3744,50,ZENK,09.01.16,53280,53313,53292,53325,3,2043,50,RKIK,09.01.16,53376,53407,53388,53407,139140,5091,1,1,1,BKUE - GART,113211,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,33,12,82,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)"
3,918,1,08.01.16,11653,2,2,3744,50,ZENK,08.01.16,24480,24510,24492,24510,3,2043,50,RKIK,08.01.16,24576,24586,24588,24586,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,30,0,76,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)"
4,918,1,08.01.16,11646,2,2,3744,50,ZENK,08.01.16,53280,53319,53292,53334,3,2043,50,RKIK,08.01.16,53376,53434,53388,53445,75211,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,39,15,100,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)"


In [16]:
# Drop some unwanted columns
columns_to_drop = ['halt_diva_von', 'halt_punkt_diva_von', 'halt_diva_nach', 'halt_punkt_diva_nach', 'fw_kurz']
final.drop(columns=columns_to_drop, inplace=True)

# Addressing the signal intervals:

    Dealing with granularity issues:

In [17]:
# reformat the date
d = pd.to_datetime(final['betriebsdatum'], format='%d.%m.%y', dayfirst=True)
# Monday == 0 … Sunday == 6
final['day_of_week'] = [i.weekday() for i in d]

In [18]:
# Remove type 2 - irrelevant 
final = final[final.fw_typ != 2]

In [19]:
# Make sure the GPS data is in order
list(set(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x)))[:5]

[(47.30712, 8.548644),
 (47.38303, 8.499871),
 (47.395936, 8.60021),
 (47.358689, 8.527014),
 (47.370287, 8.514156)]

In [20]:
# # encode routes for easier looping
final['Route_Node_id_str'] = final.Route_Node_id_tuple.apply(lambda x: '{}_{}'.format(x[0], x[1]))
le = preprocessing.LabelEncoder()
le.fit(final.Route_Node_id_str)
final['Route_Node_id'] = le.transform(final.Route_Node_id_str)

In [21]:
# Drop Sat Sun
final = final[((final.day_of_week != 5) & (final.day_of_week != 6))]
final = final[(final['soll_an_von'] >= 21600) & (final['soll_an_von'] <= 72000)]

In [22]:
# Total road segments
len(set(final.Road_Segment_Node))

1583

In [23]:
final.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple,day_of_week,Route_Node_id_str,Route_Node_id
0,918,1,06.01.16,11652,1,2,ZENK,06.01.16,65880,65906,65898,65906,3,RKIK,06.01.16,65988,65997,66000,65997,75165,5091,1,1,BKUE - GART,112395,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,26,0,91,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)",2,2663_2796,1211
1,918,1,06.01.16,11664,2,2,ZENK,06.01.16,24480,24561,24492,24572,3,RKIK,06.01.16,24576,24657,24588,24657,75179,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,81,11,85,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)",2,2663_2796,1211
3,918,1,08.01.16,11653,2,2,ZENK,08.01.16,24480,24510,24492,24510,3,RKIK,08.01.16,24576,24586,24588,24586,75179,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,30,0,76,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)",4,2663_2796,1211
4,918,1,08.01.16,11646,2,2,ZENK,08.01.16,53280,53319,53292,53334,3,RKIK,08.01.16,53376,53434,53388,53445,75211,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,39,15,100,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)",4,2663_2796,1211
5,918,1,08.01.16,11653,2,2,ZENK,08.01.16,42480,42497,42492,42508,3,RKIK,08.01.16,42576,42591,42588,42591,75199,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,17,11,83,47.318319,8.581964,47.316607,8.583933,"(47.31831928673985, 8.581964332468907)","(47.316606999997035, 8.583933000000767)","((47.31831928673985, 8.581964332468907), (47.3...","(2663, 2796)",4,2663_2796,1211


In [24]:
len(set(final.halt_id_x))

649

In [ ]:
# Define the distance signal based on punkt to punkt
# from geopy.distance import geodesic
# # geodesic(newport_ri, cleveland_oh).miles
# final['distance'] = final.apply(lambda x: vincenty((x['GPS_Latitude_x_x'], x['GPS_Longitude_x_x']), (x['GPS_Latitude_y_x'], x['GPS_Longitude_y_x'])).meters, axis=1)

In [ ]:
# final.distance.isna().sum()

In [ ]:
# final['speed'] = final.distance/final.actual_travel_time

In [25]:
# Define the time intervals
time_period = 600
# Previously was at 5 am
starting_time = 21600
# indicates 10 mins intervals
# Number of bins:
n_bins = int(np.floor((72000 - starting_time)/time_period))
labels = [i for i in range(n_bins)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins + 1)]
final['intervals'] = pd.cut(final[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels, right=True, include_lowest=True)

In [26]:
len(labels)

84

In [27]:
final.Route_Node_id.value_counts()[:5]

1066    33302
207     33171
141     27578
381     27518
380     27357
Name: Route_Node_id, dtype: int64

In [ ]:
# final['actual_travel_time_n_target'] = final.actual_travel_time / final.target_travel_time
# final.actual_travel_time_n_target = final.actual_travel_time_n_target - final.actual_travel_time_n_target.mean()
# final.actual_travel_time_n_target_mean = final.actual_travel_time_n_target_mean - final.actual_travel_time_n_target_mean.mean()

# Define new signal: Normalized travel time - close to TTI - Version 1, normalize by target travel time

In [28]:
final['actual_travel_time_n_target'] = final.actual_travel_time.div(final.target_travel_time, axis=0)
# # final

# Define new signal: Normalized travel time - Version 2, normalize by mean of target in that interval

In [29]:
final['actual_travel_time_n_target_mean'] = final.actual_travel_time.div(final.target_travel_time.mean(), axis=0)
# # final

In [30]:
# %matplotlib
# a = final[final.Route_Node_id == 1067]
# a = a[a.betriebsdatum == '08.01.16']
# # a = a.sort_values(by=target_departure_from)
# a = a.groupby(by='intervals', as_index=False).mean()
# # a.intervals.value_counts()
# # a = a.groupby(by='intervals', as_index=False).mean()
# # a.fillna(value=0)
# # plt.plot(a[target_departure_from], a.target_travel_time)
# plt.plot(a.intervals.values.tolist(), a.actual_travel_time_n_target.values.tolist(), a.intervals.values.tolist(), a.actual_travel_time_n_target.values.tolist())

In [31]:
# a = final[final.Route_Node_id == 1067]
# a = a[a.betriebsdatum == '08.01.16']
# for i in set(a.fahrweg_id):
#     b = a[a.fahrweg_id == i]
#     b.sort_values(by=['intervals', target_departure_from], inplace=True)
#     if b.shape[0] < 50:
#         continue
#     plt.plot(b[target_departure_from], b.actual_travel_time_n_target)
# a = a.groupby(by='intervals', as_index=False).mean()
# plt.title('Signal')
# plt.xlabel('Times of the day')
# plt.ylabel('')
# a.sort_values(by=['intervals', target_departure_from], inplace=True)
# plt.scatter([time_period * i + 21600 for i in a.intervals.values.tolist()], np.array(a.actual_travel_time_n_target.values.tolist()), marker='^', s=1000)

In [32]:
# # Road seg in the mid in terms of how many recordings there are
# final.Route_Node_id.value_counts()[200:205]

In [33]:
# Find the Nodes with gran. problems:

list_road_seg = final.Route_Node_id.value_counts()
list_road_seg_id = []
list_empty_bins_index = []
list_to_drop = []

for index, item in list_road_seg.items():
    temp = final[final.Route_Node_id == index]
    if (0 in (temp.intervals.value_counts().value_counts())):
        list_road_seg_id.append(index)
        list_empty_bins_index.append(temp.intervals.value_counts().value_counts()[0])
        list_to_drop.append(index)
    else:
        list_road_seg_id.append(index)
        list_empty_bins_index.append(0)
list_empty_bins_index = np.array(list_empty_bins_index)

In [34]:
# Number of road segments where they have some empty bins
1560 - len(set(list_to_drop))

741

In [35]:
# How many are actually empty
# list_empty_bins_index

In [36]:
%matplotlib
len(list_empty_bins_index)
pd.Series(list_empty_bins_index).hist()
plt.title('Distribution of non empty bins - 10 mins - Start at 6 AM - 8 PM')
plt.xlabel('Number of empty bins')
plt.ylabel('Number of full bins')

Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Number of full bins')

In [37]:
%matplotlib
# Plot the halts vs. empty bins
plt.scatter(list_road_seg_id, list_empty_bins_index)
plt.title('Scatter of empty and non empty bins - 10 mins  - Start at 6 - 8')
plt.xlabel('Road ID')
plt.ylabel('Number of empty bins')


Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Number of empty bins')

In [38]:
# How many missing below 6 - how many exactly 0 
print(np.count_nonzero(list_empty_bins_index < 6))
print(np.count_nonzero(list_empty_bins_index == 0))

858
764


# A Clear Issue with granularity
solving it by limiting the start date and visualizing the distribution


In [39]:
# rev_dist_bins = pd.Series(index = final.intervals.value_counts().values, data=final.intervals.value_counts().index.values)
plt.scatter(final.intervals.value_counts().index.values, final.intervals.value_counts().values)
plt.title('scatter - bin number vs data count - start at 6, end at 8 - 10 mins')
plt.xlabel('Bin number')
plt.ylabel('Count of Data points')

Text(0, 0.5, 'Count of Data points')

In [40]:
# top bins with the most number of recordings
final.intervals.value_counts()[:5]

10    139137
7     137804
11    136841
9     136720
6     136656
Name: intervals, dtype: int64

In [41]:
# Drop those datapoints:
print(np.count_nonzero(list_empty_bins_index != 0))
len(list_to_drop)

819


819

In [42]:
len(final)

10245941

In [43]:
final = final[~final.Route_Node_id.isin(list_to_drop)]

In [44]:
len(final)

7845361

In [45]:
# most recordings per route
final.Route_Node_id.value_counts()[:5]

1066    33302
207     33171
141     27578
381     27518
380     27357
Name: Route_Node_id, dtype: int64

# Create Signal

In [46]:
# Creating Signals for all of the nodes:
List_of_Nodes = [str(i) for i in list(set(final.Route_Node_id))]

In [47]:
len(List_of_Nodes)

764

In [48]:
List_of_head_data = {}
for i in List_of_Nodes:
    temp_df = final[final.Route_Node_id == int(i)]
#     temp_df = temp_df.sort_values(by=['betriebsdatum'])
    time_series_for_node = []
    for j in sorted(list(set(temp_df.betriebsdatum)), key=lambda x: (x[3], x[4], x[0], x[1])):
#         print(j)
        time_series = temp_df[temp_df.betriebsdatum == j]
        time_series = time_series.groupby(by='intervals', as_index=False).mean()
        time_series = time_series['actual_travel_time_n_target']
        time_series_for_node.append(time_series)
    final_series = pd.concat(time_series_for_node, ignore_index=True)
    List_of_head_data[i] = final_series

In [49]:
# count nans in the signals
new_dict = {}
for key, value in List_of_head_data.items():
    new_dict[key] = value.isna().sum()

In [ ]:
new_dict

In [50]:
# Finding dicts with distribution
nan_dist = pd.Series(new_dict)
nan_dist.hist()

In [51]:
# find and drop ones with 15 or more missing values
final_data_df = pd.DataFrame.from_dict(List_of_head_data)
print(sorted(final_data_df.isna().sum()))
final_data_df.dropna(thresh=len(final_data_df) - 15, axis=1, inplace=True)
# How many nodes are left?
print(len(final_data_df.columns))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13,

In [52]:
# How the nans are distributed
final_data_df.isna().sum().hist()

In [53]:
# How many are truly zero
final_data_df.isna().sum().value_counts()[0]

122

In [54]:
print(sorted(final_data_df.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13,

In [55]:
# How many total nans are present
final_data_df.isna().sum().sum()

1577

In [ ]:
final_data_df.head()

In [56]:
# Interpolate the missing values
final_data_df = final_data_df.interpolate(axis=1, limit=16)

In [60]:
# How many nans are left
nan_counts = final_data_df.isna().sum().sum()
nan_counts

14

2

In [61]:
if nan_counts:
    # drop the column
    final_data_df.dropna(how='any', axis=1)
    # Find which column has the nan
    toDrop = final_data_df[final_data_df.isna() == True].index[0]
    print(toDrop)
    final_data_df.drop(str(toDrop), axis=1, inplace=True)
    
    

<class 'int'>


KeyError: "['0'] not found in axis"

In [67]:
final_data_df.drop(str(8), axis=1, inplace=True)

In [68]:
# Make sure it is dropped
final_data_df.isna().sum().sum()

0

In [69]:
# Visualize some of the signals
final_data_df.columns.values.tolist()[20:25]

['79', '87', '88', '90', '94']

In [71]:
sig_1 = final_data_df['79']
sig_2 = final_data_df['87']

In [72]:
plt.plot(final_data_df.index, sig_1, final_data_df.index, sig_2)

# Signal is Defined, Create the graphs in two ways! Using the signal DF!

# Create The graph - V1

In [73]:
final['Mid_point_GPS_data'] = final.Road_Segment_Node.apply(lambda x: (((x[0][0] + x[1][0])/2) , ((x[0][1] + x[1][1])/2)))

In [74]:
# How many nodes left
len(final_data_df.columns.values.tolist())

336

In [75]:
# List of road segments by id
list_of_nodes_Road = list(set(final_data_df.columns.values.tolist()))

In [76]:
from geopy.distance import vincenty
from geopy.distance import geodesic
from networkx.drawing.nx_pydot import write_dot

In [77]:
# Edit final to get the most updated gps data
list_of_nodes_Road_toKeep = [int(i) for i in list_of_nodes_Road]
final = final[final.Route_Node_id.isin(list_of_nodes_Road_toKeep)]
# final

In [78]:
# Create List of dicts:

new_final = final.groupby(by='Route_Node_id', as_index=False).min()

In [79]:
Dict_List = pd.Series(new_final.Mid_point_GPS_data.values,index=new_final.Route_Node_id).to_dict()
Dict_List

{11: (47.368752703332575, 8.493530470205895),
 12: (47.36872096062622, 8.546983495075853),
 13: (47.371504519533524, 8.549670195002955),
 15: (47.3716133726147, 8.548133486524543),
 16: (47.369850061287835, 8.551909616283595),
 17: (47.37380228521639, 8.527599728970475),
 18: (47.37427435579169, 8.523351453003245),
 20: (47.37871457649162, 8.548136735294445),
 21: (47.37839095680438, 8.54636382616415),
 25: (47.398058327746476, 8.537514621706384),
 27: (47.40020157240362, 8.534157704769658),
 28: (47.39664437010312, 8.53477825884045),
 32: (47.39813237305747, 8.530987450724206),
 40: (47.4106918770685, 8.545633117476566),
 41: (47.41042516186653, 8.549012477953312),
 42: (47.41171715294287, 8.54623230841747),
 44: (47.408120317879224, 8.547427080037421),
 67: (47.367053845170844, 8.561292178828229),
 68: (47.36512069814996, 8.564302552654015),
 76: (47.41615363131163, 8.552645120945822),
 79: (47.41332423879862, 8.482206810699829),
 87: (47.3318209640325, 8.51701167377289),
 88: (47.33

In [80]:
# Find edges and weights:
list_of_edges = []

# iterate over each mid point aka road segment
for i in list_of_nodes_Road:
    # Look at every other road segment, if physical distance was less than 2.5 kilometers, add an edge
    
    for key, value in Dict_List.items():
        if Dict_List[int(i)] != value:
            if vincenty(Dict_List[int(i)], value).meters < 1000:
                list_of_edges.append((int(i), key, 10/(vincenty(Dict_List[int(key)], Dict_List[int(i)]).kilometers)))
    
#     a = new_final[new_final.Route_Node_id == int(i)]
#     a.Route_Node_id_tuple.iloc[5]
#     for j in list_of_nodes:
#         if i[0] != j[0]:
#             if vincenty(i[1], j[1]).meters < 1000:
#                 list_of_edges.append((i[0], j[0]))

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # This is added back by InteractiveShellApp.init_path()


In [81]:
# Edge count by the node id
zlst = list(zip(*list_of_edges))
edge_series = pd.Series(zlst[1], index = zlst[0])
edge_series.value_counts()

99      71
541     71
298     70
381     70
532     67
144     67
1381    67
1528    67
1197    67
543     67
180     65
1369    65
380     65
343     65
141     65
1370    65
1380    65
182     64
531     64
342     63
1169    63
184     63
1022    63
179     63
544     63
337     62
1196    62
102     62
181     62
297     61
        ..
68       7
1105     7
1410     7
803      7
1478     7
90       6
401      6
868      6
94       6
1008     6
1249     6
1200     6
735      6
801      6
1267     6
1401     6
1265     5
734      5
790      5
802      5
263      5
264      5
282      5
973      4
118      4
866      4
512      1
79       1
1522     1
11       1
Length: 336, dtype: int64

In [82]:
# Define Graph
G = nx.Graph()

In [83]:
G.add_nodes_from(list_of_nodes_Road_toKeep)

In [84]:
len(set(list_of_edges))

9984

In [85]:
list_of_edges[:5]

[(41, 40, 38.9463878152347),
 (41, 42, 39.32680859684956),
 (41, 44, 35.3594634901815),
 (41, 76, 14.422061982583193),
 (41, 150, 10.31864278343454)]

In [86]:
G.add_weighted_edges_from(list_of_edges)

In [87]:
# # Visualize Graph
nx.draw_networkx(G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [88]:
len(list(G.nodes))

336

In [89]:
# Corr. based on edges

In [92]:
# # 459 is corr!
sig_1 = final_data_df['87']
# 619! 607!
list_of_node_crosscoef = []
for i in edge_series[87]:
    sig_2 = final_data_df[str(i)]
#     sig2 = sig2[sig2.betriebsdatum == '11.01.16']
#     sig2.sort_values(by='soll_an_von', inplace=True)
#     sig2 = sig2.groupby(by='intervals', as_index=False).mean()
#     sig2_y = np.array(sig2.actual_travel_time_n_target)
    list_of_node_crosscoef.append((i, np.corrcoef(sig_1, sig_2)[0][1]))
zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()
plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

In [93]:
# Save the gps data for further use

In [94]:
dict_gps = pd.Series(final.Mid_point_GPS_data.values,index=final.Route_Node_id).to_dict()
# print(len(dict_gps))
for i, v in dict_gps.items():
    if i not in list_of_nodes_Road_toKeep:
        del dict_gps[i]
dict_gps

{307: (47.374275654184025, 8.523351124082808),
 97: (47.3738027450908, 8.527599026459598),
 309: (47.37495254256646, 8.519264959689792),
 528: (47.376094067270586, 8.515597337919838),
 308: (47.374548556609994, 8.52274950488366),
 918: (47.37540930989158, 8.525080003483815),
 311: (47.373059429782224, 8.518696483806384),
 1598: (47.37096246155436, 8.515083313014506),
 199: (47.37067451160515, 8.521566260832588),
 1595: (47.370145394938675, 8.516529478597292),
 198: (47.371339467481874, 8.523955435738705),
 423: (47.37067429407692, 8.521568330644364),
 155: (47.37194253843056, 8.525418176404282),
 197: (47.37194276858143, 8.525416070707013),
 98: (47.373037711049584, 8.528240361640385),
 102: (47.372907603444744, 8.532161877459233),
 144: (47.3709840324442, 8.536847974130387),
 99: (47.37485134337582, 8.531948414267125),
 1022: (47.3772964025813, 8.53222413671795),
 532: (47.37605993484516, 8.535848632711852),
 524: (47.375409313709795, 8.525079993632776),
 1187: (47.37791890611014, 8.5

In [95]:
# GPS data df
d = pd.DataFrame(dict_gps)
d.to_csv('gps_10min_Final.csv', index=False)

In [96]:
# Save Final DF as well
final_data_df.to_csv('signal_10min_Final.csv', index=False)

In [97]:
# Shape of final df
final_data_df.shape

(5040, 336)

# Finding correlation coefficient

In [101]:
sig1 = final[final.Route_Node_id == 87]
sig1 = sig1[sig1.betriebsdatum == '07.01.16']
sig1.sort_values(by='soll_an_von', inplace=True)
# sig1 = sig1
sig1 = sig1.groupby(by='intervals', as_index=False).mean()
# sig1

In [102]:
sig2 = final[final.Route_Node_id == 88]
sig2 = sig2[sig2.betriebsdatum == '07.01.16']
sig2.sort_values(by='soll_an_von', inplace=True)
sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# sig2

In [103]:
sig1_X = np.array(sig1.intervals)
sig1_y = np.array(sig1.actual_travel_time_n_target)
sig1_y_zeromean = sig1_y - sig1_y.mean()

sig2_X = np.array(sig2.intervals)
sig2_y = np.array(sig2.actual_travel_time_n_target)
sig2_y_zeromean = sig2_y - sig2_y.mean()

plt.plot(sig1_X, sig1_y, sig2_X, sig2_y)

In [104]:
plt.plot(sig1_X, sig1_y_zeromean, sig2_X, sig2_y_zeromean)

In [105]:
from scipy.stats.stats import pearsonr
np.corrcoef(sig1_y_zeromean, sig2_y_zeromean)

array([[1.        , 0.04247711],
       [0.04247711, 1.        ]])

In [106]:
pearsonr_coef = pearsonr(sig1_y, sig2_y)
pearsonr_coef

(0.04247711457554691, 0.7012385676512146)

In [107]:
edge_series.value_counts()[:5]

99     71
541    71
298    70
381    70
532    67
dtype: int64

In [108]:
zlst = list(zip(*list_of_edges))
edge_series = pd.Series(zlst[1], index = zlst[0])
edge_series.value_counts()

99      71
541     71
298     70
381     70
532     67
144     67
1381    67
1528    67
1197    67
543     67
180     65
1369    65
380     65
343     65
141     65
1370    65
1380    65
182     64
531     64
342     63
1169    63
184     63
1022    63
179     63
544     63
337     62
1196    62
102     62
181     62
297     61
        ..
68       7
1105     7
1410     7
803      7
1478     7
90       6
401      6
868      6
94       6
1008     6
1249     6
1200     6
735      6
801      6
1267     6
1401     6
1265     5
734      5
790      5
802      5
263      5
264      5
282      5
973      4
118      4
866      4
512      1
79       1
1522     1
11       1
Length: 336, dtype: int64

In [110]:
%matplotlib
sig1 = np.array(final_data_df[str(88)])

list_of_node_crosscoef = []

for i in edge_series[88]:
    sig2 = np.array(final_data_df[str(i)])
    list_of_node_crosscoef.append((i, np.corrcoef(sig1, sig2)[0][1]))

zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()

plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())
plt.xlabel('Road ID')
plt.ylabel('pearson correlation coefficient')
plt.title('Connected nodes - G1')

Using matplotlib backend: Qt5Agg


Text(0.5, 1.0, 'Connected nodes - G1')

In [111]:
# # Autocorrelation
# def autocorr(x, t=1):
#     return np.corrcoef(np.array([x[:-t], x[t:]]))
# autocorr(sig2_y_zeromean)

In [112]:
one_day = final_data_df[:84]
%matplotlib
import seaborn as sns
corr = one_day.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

Using matplotlib backend: Qt5Agg


In [114]:
final_data_df.corr().unstack().sort_values(ascending=False).drop_duplicates()[:50]

1598  1598    1.000000
111   701     0.935669
703   652     0.879686
402   652     0.839950
653   402     0.774383
564   552     0.772696
703   402     0.770201
401   652     0.714158
90    402     0.711618
912   161     0.704232
887   713     0.699510
653   652     0.698010
703   653     0.681192
      401     0.679390
1465  1304    0.667922
402   401     0.648713
557   564     0.638023
90    652     0.625863
714   143     0.612512
90    653     0.589061
703   90      0.583884
401   90      0.565514
      653     0.542234
475   1370    0.539954
44    331     0.531126
475   1369    0.524536
913   1304    0.515936
1342  220     0.497076
88    104     0.494698
316   1137    0.488089
276   274     0.483128
1201  1477    0.477139
557   552     0.468508
1369  1370    0.465273
1381  1558    0.460906
1226  1305    0.445431
1350  1342    0.442616
1489  1106    0.439272
1401  1489    0.426144
1380  1370    0.424470
562   173     0.423132
1342  912     0.408046
12    1558    0.407260
1137  562  

In [115]:
sig1 = final[final.Route_Node_id == 549]
sig1 = sig1[sig1.betriebsdatum == '07.01.16']
sig1.sort_values(by='soll_an_von', inplace=True)
# sig1 = sig1
sig1 = sig1.groupby(by='intervals', as_index=False).mean()
# sig1
sig2 = final[final.Route_Node_id == 561]
sig2 = sig2[sig2.betriebsdatum == '07.01.16']
sig2.sort_values(by='soll_an_von', inplace=True)
sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# sig2

sig1_X = np.array(sig1.intervals)
sig1_y = np.array(sig1.actual_travel_time_n_target)
sig1_y_zeromean = sig1_y - sig1_y.mean()

sig2_X = np.array(sig2.intervals)
sig2_y = np.array(sig2.actual_travel_time_n_target)
sig2_y_zeromean = sig2_y - sig2_y.mean()

plt.plot(sig1_X, sig1_y_zeromean, sig2_X, sig2_y_zeromean)

In [116]:
# Save this graph

In [117]:
len(final_data_df.columns)

336

In [118]:
a = [int(i) for i in final_data_df.columns.values.tolist()]

adj_mat = nx.to_numpy_matrix(G, nodelist=a)

np.savetxt("PeMSD7_W_336_V1.csv", adj_mat, delimiter=",")

In [119]:
final_data_df.to_csv('PeMSD7_V_336_V1.csv', index=False)

# Create the graph another way! V2

In [120]:
Nodes = list(set(new_final.Route_Node_id))

In [121]:
Dict_List_halt_ids = pd.Series(new_final.Route_Node_id_tuple.values,index=new_final.Route_Node_id).to_dict()
len(Dict_List)

336

In [122]:
# Create the new graph:

# GPS is in Dict_List
Edges = []
for key, value in Dict_List_halt_ids.items():
    for key_2, value_2 in Dict_List_halt_ids.items():
        if key != key_2:
#             if value[0] == value_2[0] or value[0] == value_2[1] or value[1] == value_2[0] or value[1] == value_2[1]: 
            if value[0] == value_2[0] or value[1] == value_2[1]:    
                Edges.append((int(key), int(key_2), 10/(vincenty(Dict_List[int(key)], Dict_List[int(key_2)]).kilometers)))
    


C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # Remove the CWD from sys.path while we load stuff.


In [123]:
len(Edges)

872

In [124]:
New_G = nx.Graph()
New_G.add_nodes_from(list_of_nodes_Road_toKeep)
New_G.add_weighted_edges_from(Edges)

In [125]:
len(list(New_G.nodes))

336

In [126]:
nx.draw_networkx(New_G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [127]:
set1 = set(list(New_G.node))
set2 = set(new_final.Route_Node_id)
set2 - set1

set()

In [129]:
a = [int(i) for i in final_data_df.columns.values.tolist()]

adj_mat = nx.to_numpy_matrix(New_G, nodelist=a)

np.savetxt("PeMSD7_W_336_V2.csv", adj_mat, delimiter=",")

In [130]:
zlst = list(zip(*(list(New_G.edges()))))
edge_series = pd.Series(zlst[1], index = zlst[0])
edge_series.value_counts()

25      7
181     6
209     5
468     5
452     5
1169    5
99      5
130     5
106     5
184     5
921     4
108     4
555     4
40      4
309     4
15      4
399     4
1318    4
202     4
135     4
342     4
1044    4
141     4
472     4
713     4
913     3
307     3
159     3
155     3
98      3
       ..
1208    1
1336    1
1395    1
1201    1
1394    1
866     1
258     1
964     1
1150    1
143     1
131     1
137     1
337     1
1359    1
333     1
331     1
148     1
150     1
911     1
330     1
1349    1
161     1
1344    1
1190    1
1197    1
789     1
554     1
316     1
1339    1
1137    1
Length: 224, dtype: int64

In [134]:
%matplotlib
sig1 = np.array(final_data_df[str(90)])

list_of_node_crosscoef = []

for i in edge_series[90]:
    sig2 = np.array(final_data_df[str(i)])
    list_of_node_crosscoef.append((i, np.corrcoef(sig1, sig2)[0][1]))

zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()

plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())
plt.xlabel('Road ID')
plt.ylabel('pearson correlation coefficient')
plt.title('Connected nodes - G2')

Using matplotlib backend: Qt5Agg


TypeError: 'numpy.int64' object is not iterable

# Remove nodes with no connections


In [135]:
len(New_G.nodes)

336

In [136]:
Connected_New_G = max(nx.connected_component_subgraphs(New_G), key=len)
nx.draw_networkx(Connected_New_G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
# set_1 = set(list(G.nodes))
# set_2 = set([int(i) for i in list(Connected_New_G.nodes)])
# set_3 = set_1.difference(set_2)

# # Nodes to drop
# nodes_drop_to_equalGraphs = list(set_3)

# len(nodes_drop_to_equalGraphs)

# some_G = G.subgraph([int(i) for i in list(Connected_New_G.nodes)])

# nx.draw_networkx(some_G)

# len(set(final.betriebsdatum))

# # Save Files!
# Connected_final_df.shape[1]

# a = [int(i) for i in Connected_final_df.columns.values.tolist()]

# adj_mat = nx.to_numpy_matrix(some_G, nodelist=a)

# np.savetxt("PeMSD7_W_5min_V1.csv", adj_mat, delimiter=",")

# # Save the connected final df as well

# Connected_final_df.to_csv('final_5min_final_1months_233.csv')
# df = pd.read_csv('final_5min_final_1months_233.csv', nrows=252)
# # df

# d = pd.DataFrame(dict_gps)
# for i in nodes_drop_to_equalGraphs:
#     del d[i]
# d.to_csv('gps_10min_final_3months_233.csv')
# # d.columns.tolist() - [int(i) for i in Connected_final_df.columns.tolist()]
# def intersection(lst1, lst2): 
#     lst3 = [value for value in lst1 if value in lst2] 
#     return lst3
# len(intersection(d.columns.tolist(), [int(i) for i in Connected_final_df.columns.tolist()]))

In [138]:
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

import folium
import folium.plugins as plugins
import numpy as np

df = final_data_df

print(df.isna().sum().sum())

# gps data
df_2 = pd.read_csv('gps_10min_Final.csv')
# df_2 = d

df = (df - df.mean()) / (df.std() ** 1.2)

# df_2.drop('Unnamed: 0', axis=1, inplace=True)
# df.drop('Unnamed: 0', axis=1, inplace=True)

df = df[:336]

# Outer list:
list_df_gps = df_2.columns.values.tolist()
OuterList = []
for index, row in df.iterrows():
    Mid_List = []
    for j in list_df_gps:
#         print(j)
#         if (j not in df_2.columns.values.tolist()):
#             continue
        Time_List = df_2[j].values.tolist()
#         print(type(j))
#         print(row[0])
        Time_List.append(row[j])
        Mid_List.append(Time_List)
    OuterList.append(Mid_List)

m = folium.Map([47.36179377478453, 8.572671632662107], tiles='stamentoner', zoom_start=12)

list_of_id = df_2.values
list_of_id = list(zip(list_of_id[0], list_of_id[1]))

for j in list_of_id:
    folium.Circle(j, popup='<strong>Location One</strong>', radius=20).add_to(m)

index = [str(timedelta(seconds=i * time_period + starting_time)) for i in set(final.intervals)] * int(df.shape[0]/len(set(final.intervals)))

hm = plugins.HeatMapWithTime(OuterList, index=index)

hm.add_to(m)

m.save('HeatMap_moving.html')

0


# Save Files

In [ ]:
# a = [int(i) for i in final_data_df.columns.values.tolist()]

# adj_mat = nx.to_numpy_matrix(New_G, nodelist=a)

In [ ]:
# Connected_final_df.to_csv('PeMSD7_V_233.csv', header=False, index=False)

In [ ]:
# np.savetxt("PeMSD7_W_233_V2.csv", adj_mat, delimiter=",")

In [ ]:
# len(list(New_G.nodes))

In [ ]:
# New_G.edges(data=True)

In [ ]:
# Connected_final_df.shape